- DI is easy to do in Python due to its support for keyword arguments, the ease with which objects can be mocked and its dynamic nature, and a framework assisting in this process can remove a lot of boiler-plate from larger apps.
- That's where Injector can help. It automatically and transitively provides dependencies for you. As an added benefit, Injector encourages nicely compartmentalised code through the use of :ref:modules <module>

The core values of Injector are:
- Simplicity - while being inspired by Guice, Injector does not slavishly replicate its API. Providing a Pythonic API trumps faithfulness. Aditionally some features are omitted because supporting them would be cumbersome and introduce a little bit too much "magic" (member injection, method injection)

- No gloval state: you can have as many Injector instances as you like, each with a diferent configuration and each with different objects in different scopes.

## Quick example

In [1]:
from injector import Injector, inject, Module, provider, singleton
from dataclasses import dataclass

In [2]:
@dataclass
class Inner:
    a_number: int = 42
    # def __init__(self, num):
    #    self.a_number = num
        

In [3]:
@inject
@dataclass
class Outer_dummy:
    inner: Inner
        
#@inject
#@dataclass
#class Outer:
#    inner: Inner

In [4]:
injector = Injector()

flag = True

if flag :
    outer = injector.get(Outer_dummy)
else:
    pass
    outer = injector.get(Outer)
    
print(outer.inner.a_number)

42


In [5]:
import sqlite3


In [6]:
class RequestHandler:
    @inject
    def __init__(self, db: sqlite3.Connection):
        self._db = db
    
    def get(self):
        cursor = self._db.cursor()
        cursor.execute('SELECT key, value FROM data ORDER BY key')
        return cursor.fetchall()

class Configuration:
    def __init__(self, conn_string):
        self.connection_string = conn_string
    

In [7]:
def configure_for_testing(binder):
    configuration = Configuration(':memory:')
    binder.bind(Configuration, to=configuration, scope=singleton)

In [8]:
class DatabaseModule(Module):
    @singleton
    @provider
    def provide_sqlite_connection(self, configuration: Configuration) -> sqlite3.Connection:
        conn = sqlite3.connect(configuration.connection_string)
        cursor = conn.cursor()
        cursor.execute('CREATE TABLE IF NOT EXISTS data (key PRIMARY KEY, value)')
        cursor.execute('INSERT OR REPLACE INTO data VALUES ("hello", "world")')
        return conn

In [9]:
injector = Injector([configure_for_testing, DatabaseModule])
handler = injector.get(RequestHandler)
handler.get()

[('hello', 'world')]

## Injection
- Injection is the process of providing an instance of a type, to a method that uses that instance. It is achieved with the inject decorator. Keyword arguments to inject define which arguments in its deccorated method should be injected, and with what.
- Here is an example of injection on a module provider method, and on the constructor of a normal class:

## Injector
- The *Injector* brings everthing together. It takes a list of Moudles, and configures them with a binder, effectively creating a depndency graph: 

In [10]:
from injector import Injector


In [11]:
class Name:
    name: str

class Description:
    description: str
        
class User:
    @inject
    def __init__(self, name: Name, description: Description):
        self.name = name
        self.description = description

class UserModule(Module):
    def configure(self, binder):
       binder.bind(User)


class UserAttributeModule(Module):
    def configure(self, binder):
        binder.bind(Name, to='Sherlock')

    @provider
    def describe(self, name: Name) -> Description:
        return '%s is a man of astounding insight' % name

In [12]:
injector = Injector([UserModule(), UserAttributeModule])

In [13]:
injector.get(Name)

'Sherlock'

In [14]:
injector.get(Description)

'Sherlock is a man of astounding insight'

In [15]:
user = injector.get(User)
isinstance(user, User)

True

In [16]:
user.name

'Sherlock'

In [17]:
user.description

'Sherlock is a man of astounding insight'

### Injection
- At its heart, Injector is simply a dictionary for mapping types to things that create instances of those type.This could be as simple as:

```
{mystr: 'an instance of a string'}
```

For those new to dependency injection and/or Guice, though, some of the terminology used may not be obvious.

### Provider
A means of providing an instance of a type. Built-in providers include:
- Class Provider
- Instance Provider
- Callable Provider
In order to create custom provider you need to to subclass `Provider` and override it `get()` method.

### Scope

By default, providers are executed each time an instance is required. Scopes allow this behaviour to be customised. For example, Singleton Scope (typically used through the class decorator singleton), can be used to always provide the same instance of a class.

Other examples of where scopes might be a threading scope, where instances are provided per-thread, or a request scope, where instances are provided per-HTTP request.

The default scope is `NoScope`.

### Binding

A binding is the mapping of a unique binding key to a coresponding provider. For example:


In [21]:
from injector import InstanceProvider
bindings = {
    (Name, None): InstanceProvider('Sherlock'),
    (Description, None): InstanceProvider('A man of astounding insight')
}

In [22]:
bindings

{(__main__.Name, None): InstanceProvider('Sherlock'),
 (__main__.Description, None): InstanceProvider('A man of astounding insight')}

In [24]:
str(Name)

"<class '__main__.Name'>"

### Binder

The `Binder` is simply a convenient wrapper around the dictionary that maps types to providers. It provides methods that make declaring bindings easier.

### Module

A `Module` configure bindings. It provides methods that simplify the process of bindings a key to a provider. For example the above bindings would be created with:

For more complex instance construction, methods decorated with `@provider` will called to resolve binding keys:

In [25]:
from injector import singleton


In [33]:
@singleton
class TodoUseCase:
    todo : str
    
    def register(self, todo: str) -> None:
        self.todo = todo
        print(f'My todo = "{todo}"')

In [38]:
@singleton
class TodoController:
    todo_uc : TodoUseCase
    @inject
    def __init__(self, todo_uc: TodoUseCase) -> None:
        self.todo_uc = todo_uc
        
    def create(self, todo: str) -> None:
        self.todo_uc.register(todo)

In [39]:
injector = Injector()
todo_controller : TodoController = injector.get(TodoController)
todo: str = 'Activity of Coding'

todo_controller.create(todo)

My todo = "Activity of Coding"
